In [1]:
from zernike_algos import (
    ZernikePolynomial,
    zernike_radial,
    #zernike_radial_dumb,
    zernike_radial_optimized,
)
import numpy as np

DESC version 0.10.2+253.g43c94ecb.dirty,using JAX backend, jax version=0.4.19, jaxlib version=0.4.14, dtype=float64
Using device: CPU, with 983.44 GB available memory


In [2]:
basis = ZernikePolynomial(L=50, M=50, spectral_indexing="ansi", sym="cos")
r = np.linspace(0, 1, 100)
radial = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 2)
print(radial)

[[ 0.00000000e+00  0.00000000e+00  4.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00 ...  3.63270541e-82
  -1.75495958e-87  3.96896020e-93]
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00 ...  6.38627794e-69
  -1.23452408e-73  1.11716298e-78]
 ...
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00 ...  2.87059206e+03
   2.75975270e+03  9.19856303e+02]
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00 ...  1.10757887e+04
   5.95138956e+03  1.50497185e+03]
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00 ...  3.00980000e+04
   1.19560000e+04  2.45000000e+03]]


In [3]:
print("zernike_radial, 0th derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 0).block_until_ready()
print("zernike_radial, 1st derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 1).block_until_ready()
print("zernike_radial, 2nd derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 2).block_until_ready()
print("zernike_radial, 3rd derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 3).block_until_ready()

zernike_radial, 0th derivative
12.6 ms ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
zernike_radial, 1st derivative
12.6 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
zernike_radial, 2nd derivative
12.6 ms ± 19.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
zernike_radial, 3rd derivative
12.7 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
print("zernike_radial, 0th derivative")
%timeit _ = zernike_radial_optimized(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 0).block_until_ready()
print("zernike_radial, 1st derivative")
%timeit _ = zernike_radial_optimized(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 1).block_until_ready()
print("zernike_radial, 2nd derivative")
%timeit _ = zernike_radial_optimized(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 2).block_until_ready()
print("zernike_radial, 3rd derivative")
%timeit _ = zernike_radial_optimized(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 3).block_until_ready()

zernike_radial, 0th derivative
3.69 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
zernike_radial, 1st derivative
7.92 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
zernike_radial, 2nd derivative
10.9 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
zernike_radial, 3rd derivative
14.8 ms ± 449 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
